# CUDA的动态并行性
创建一个基于N个线程的内核函数，令每个线程向终端输出一条短消息，然后递归地启动另一个基于N-1个线程的内核函数

In [1]:
from __future__ import division
import numpy as np
from pycuda.compiler import DynamicSourceModule
import pycuda.autoinit

DynamicParallelismCode='''
__global__ void dynamic_hello_ker(int depth)
{
 printf("Hello from thread %d, recursion depth %d!\\n", threadIdx.x, depth);
 if (threadIdx.x == 0 && blockIdx.x == 0 && blockDim.x > 1)
  {
   printf("Launching a new kernel from depth %d .\\n", depth);
   printf("-----------------------------------------\\n");
   dynamic_hello_ker<<< 1, blockDim.x - 1 >>>(depth + 1);
  }
}'''

dp_mod = DynamicSourceModule(DynamicParallelismCode)

hello_ker = dp_mod.get_function('dynamic_hello_ker')

hello_ker(np.int32(0), grid=(1,1,1), block=(4,1,1))


Hello from thread 0, recursion depth 0!
Hello from thread 1, recursion depth 0!
Hello from thread 2, recursion depth 0!
Hello from thread 3, recursion depth 0!
Launching a new kernel from depth 0 .
-----------------------------------------
Hello from thread 0, recursion depth 1!
Hello from thread 1, recursion depth 1!
Hello from thread 2, recursion depth 1!
Launching a new kernel from depth 1 .
-----------------------------------------
Hello from thread 0, recursion depth 2!
Hello from thread 1, recursion depth 2!
Launching a new kernel from depth 2 .
-----------------------------------------
Hello from thread 0, recursion depth 3!
